!pip install xdggs
!pip install healpy
!pip install -U zarr==2.18.4
!pip install numcodecs==0.13.1

In [1]:
import warnings
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xdggs
import fsspec

warnings.filterwarnings("ignore")
xr.set_options(display_max_rows=27)

In [11]:
fs = fsspec.filesystem("http")
url="/home/jovyan/Akathon-Geilo/odysea_10_nest.zarr"
url="https://data-fair2adapt.ifremer.fr/odysea/odysea.zarr"
url="https://data-fair2adapt.ifremer.fr/gebco/healpix_level_11.zarr"

ds=xr.open_zarr(url).pipe(xdggs.decode)
#dtotal=xr.open_zarr().pipe(xdggs.decode)
cell_centers = ds.dggs.cell_centers()
ds=ds.assign_coords(
    cell_centers.coords
)
ds

<xarray.Dataset> Size: 2GB
Dimensions:    (cells: 50331648)
Coordinates:
  * cell_ids   (cells) int64 403MB dask.array<chunksize=(4194304,), meta=np.ndarray>
    latitude   (cells) float64 403MB 0.01865 0.0373 0.0373 ... -0.0373 -0.01865
    longitude  (cells) float64 403MB 45.0 45.02 44.98 45.0 ... 315.0 315.0 315.0
Dimensions without coordinates: cells
Data variables:
    gebco      (cells) float64 403MB dask.array<chunksize=(4194304,), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(nside=11, indexing_scheme=nested)

In [12]:
lat = np.arange(40, 70, 0.01)
lon = np.arange(-24, 15, 0.01)
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))
ds.gebco.dggs.sel_latlon(longitude=full_lon, latitude=full_lat).compute().dggs.explore(alpha=0.8)
#

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x776aeadf…